## Comprehensive WorldView Example with `asp_plot`

This notebook demonstrates the full capabilities of `asp_plot` for visualizing NASA Ames Stereo Pipeline (ASP) processing results from WorldView satellite imagery.

The example uses WorldView-3 data from Utqiagvik, Alaska (April 17, 2022) processed with ASP.

---

### Processing Overview

This notebook covers:
1. **Full report generation** - Automated comprehensive PDF report from CLI call
2. **Processing parameters** - Extract and display ASP command history
3. **Scene visualization** - Display input satellite imagery
4. **Stereo geometry** - Analyze acquisition geometry and viewing angles
5. **Bundle adjustment** - Visualize camera optimization residuals
6. **Stereo results** - Display DEMs, hillshades, disparity maps, and match points
7. **ICESat-2 validation** - Compare DEMs with altimetry data
8. **CSM camera analysis** - Compare original and optimized camera models (Using example data from Salar de Uyuni)

Each section can be run independently for example modular analysis using the package.

## [View complete notebook with all outputs and figures](https://www.bendirt.com/asp-plot-worldview)

## [View the report generated by the `asp_plot` CLI call](https://www.bendirt.com/assets/documents/asp_plot_example_report.pdf)

## 1. Full Report Generation

The `asp_plot` CLI tool generates a comprehensive PDF report combining all visualizations below. This is the quickest way to get an overview of your stereo processing results.

In [ ]:
directory = "/Users/ben/Dropbox/UW_Shean/WV/2022/WV03_20220417_1040010074793300_1040010075633C00/"
ba_directory = "ba/"
stereo_directory = "stereo/"

In [ ]:
!asp_plot \
  --directory $directory \
  --bundle_adjust_directory $ba_directory \
  --stereo_directory $stereo_directory \
  --map_crs EPSG:32604 \
  --dem_gsd 1 \
  --subset_km 1 \
  --add_basemap True \
  --plot_icesat True \
  --plot_geometry True

---

## Individual Plots

The sections below demonstrate modular usage of `asp_plot` for detailed analysis and customization.

## 2. Processing Parameters

Extract and display the command-line parameters used for bundle adjustment, stereo processing, and DEM generation. This provides full traceability of processing settings.

In [ ]:
%load_ext autoreload
%autoreload 2

from asp_plot.processing_parameters import ProcessingParameters

In [ ]:
processing_parameters = ProcessingParameters(
    processing_directory=directory,
    bundle_adjust_directory=ba_directory,
    stereo_directory=stereo_directory
)
processing_parameters_dict = processing_parameters.from_log_files()

print(f"Processed on: {processing_parameters_dict['processing_timestamp']}\n")

print(f"Reference DEM: {processing_parameters_dict['reference_dem']}\n")

print(f"Bundle adjustment ({processing_parameters_dict['bundle_adjust_run_time']}):\n")
print(processing_parameters_dict["bundle_adjust"])

print(f"\nStereo ({processing_parameters_dict['stereo_run_time']}):\n")
print(processing_parameters_dict["stereo"])

print(f"\nPoint2dem ({processing_parameters_dict['point2dem_run_time']}):\n")
print(processing_parameters_dict["point2dem"])

## 3. Scene Plots

Visualize the input satellite imagery (orthorectified scenes) used for stereo processing. This helps verify image quality and overlap.

In [ ]:
import os
from asp_plot.scenes import ScenePlotter

In [ ]:
# Optional: create directory to save plots

# plots_directory = os.path.join(directory, "asp_plots")
# os.makedirs(plots_directory, exist_ok=True)

In [ ]:
plotter = ScenePlotter(
  directory,
  stereo_directory,
  title="Input Scenes"
)

plotter.plot_scenes(
  # Optional parameters to set for saving (requires plots_directory to be created above):

  # save_dir=plots_directory,
  # fig_fn="stereo_scenes.png"
)

## 4. Scene Geometry Plots

Analyze stereo acquisition geometry from XML camera metadata. Displays:
- **Skyplot**: Satellite viewing angles (azimuth and elevation)
- **Map view**: Scene footprints and overlap area
- **Stereo metrics**: Convergence angle, base-to-height ratio, asymmetry

In [ ]:
from asp_plot.stereo_geometry import StereoGeometryPlotter

In [ ]:
geometry_plotter = StereoGeometryPlotter(
  directory
)

geometry_plotter.dg_geom_plot(
    # Optional parameters to set for saving (requires plots_directory to be created above):

    # save_dir=plots_directory,
    # fig_fn="scene_geometry.png"
)

## 5. Bundle Adjustment Plots

Visualize bundle adjustment optimization results:
- **Residual maps**: Initial vs. final camera optimization residuals
- **Geodiff**: Triangulated point differences vs. reference DEM
- **Map-projected residuals**: Interest point separation on reference DEM surface

Bundle adjustment refines camera models to improve stereo matching consistency.

In [ ]:
import contextily as ctx
from asp_plot.bundle_adjust import ReadBundleAdjustFiles, PlotBundleAdjustFiles

In [ ]:
map_crs = "EPSG:32604"  # UTM Zone 4N

ctx_kwargs = {
    "crs": map_crs,
    "source": ctx.providers.Esri.WorldImagery,
    "attribution_size": 0,
    "alpha": 0.5,
}

In [ ]:
ba_files = ReadBundleAdjustFiles(directory, ba_directory)
resid_initial_gdf, resid_final_gdf = ba_files.get_initial_final_residuals_gdfs(residuals_in_meters=True)
resid_mapprojected_gdf = ba_files.get_mapproj_residuals_gdf()
resid_triangulation_uncert_df = ba_files.get_propagated_triangulation_uncert_df()

In [ ]:
resid_triangulation_uncert_df

### Bundle Adjustment Residuals (Log Scale)

Log scale visualization emphasizes patterns across wide value ranges.

In [ ]:
plotter = PlotBundleAdjustFiles(
  [resid_initial_gdf, resid_final_gdf],
  lognorm=True,
  title="Bundle Adjust Initial and Final Residuals (Log Scale)"
)

plotter.plot_n_gdfs(
    column_name="mean_residual",
    cbar_label="Mean residual (px)",
    map_crs=map_crs,
    **ctx_kwargs
)

plotter.plot_n_gdfs(
    column_name="mean_residual_meters",
    cbar_label="Mean residual (m)",
    map_crs=map_crs,
    **ctx_kwargs
)

### Bundle Adjustment Residuals (Linear Scale)

Linear scale shows absolute residual magnitudes more clearly.

In [ ]:
plotter.lognorm = False
plotter.title = "Bundle Adjust Initial and Final Residuals (Linear Scale)"

plotter.plot_n_gdfs(
    column_name="mean_residual",
    cbar_label="Mean residual (px)",
    common_clim=False,
    map_crs=map_crs,
    **ctx_kwargs
)

plotter.plot_n_gdfs(
    column_name="mean_residual_meters",
    cbar_label="Mean residual (m)",
    common_clim=False,
    map_crs=map_crs,
    **ctx_kwargs
)

plotter.title = "Bundle Adjust Initial and Final Residuals (Common Linear Scale)"

plotter.plot_n_gdfs(
    column_name="mean_residual",
    cbar_label="Mean residual (px)",
    map_crs=map_crs,
    **ctx_kwargs
)

### Bundle Adjustment Observation Statistics

Number of observations per point and height above datum.

In [ ]:
plotter = PlotBundleAdjustFiles(
  [resid_final_gdf],
)

plotter.title = "Bundle Adjust Residual Number of Observations"

plotter.plot_n_gdfs(
    column_name="num_observations",
    cbar_label="Number of Observations",
    map_crs=map_crs,
    **ctx_kwargs
)

plotter.title = "Bundle Adjust Residuals Height Above Datum"

plotter.plot_n_gdfs(
    column_name="height_above_datum",
    cbar_label="Height above datum (m)",
    map_crs=map_crs,
    **ctx_kwargs
)

### Map-Projected Residuals

Distance between interest points when projected onto the reference DEM surface.

In [ ]:
plotter = PlotBundleAdjustFiles(
  [resid_mapprojected_gdf],
  title="Bundle Adjust Midpoint distance between\nfinal interest points projected onto reference DEM",
)

plotter.plot_n_gdfs(
    column_name="mapproj_ip_dist_meters",
    cbar_label="Interest point distance (m)",
    map_crs=map_crs,
    **ctx_kwargs
)

### Geodiff vs. Reference DEM

Compares triangulated points (before and after bundle adjustment) to the reference DEM surface.

In [ ]:
geodiff_initial_gdf, geodiff_final_gdf = ba_files.get_initial_final_geodiff_gdfs()

In [ ]:
plotter = PlotBundleAdjustFiles(
  [geodiff_initial_gdf, geodiff_final_gdf],
  lognorm=False
)

plotter.title = "Bundle Adjust Initial and Final Geodiff vs. Reference DEM"

plotter.plot_n_gdfs(
    column_name="height_diff_meters",
    cbar_label="Height difference (m)",
    map_crs=map_crs,
    cmap="RdBu",
    clim=(-1, 1),
    **ctx_kwargs
)

plotter.title = "Bundle Adjust Initial and Final Geodiff vs. Reference DEM (Auto Scale)"

plotter.plot_n_gdfs(
    column_name="height_diff_meters",
    cbar_label="Height difference (m)",
    map_crs=map_crs,
    cmap="RdBu",
    symm_clim=True,
    **ctx_kwargs
)

## 6. Stereo Processing Results

Visualize the primary outputs of stereo processing:
- **Hillshade**: Shaded relief with intersection error overlays
- **Match points**: Interest point locations used for stereo correlation
- **Disparity**: Pixel offsets (in meters and pixels) between left/right images
- **DEM results**: Elevation, intersection error, and differences vs. reference DEM

In [ ]:
from asp_plot.stereo import StereoPlotter

In [ ]:
# Reference DEM can be passed to the StereoPlotter, but if it is not, it will
# be searched for in the stereo logs. If it's not found there, you will be warned,
# and no difference plots will be generated.

# reference_dem = "/Users/ben/Dropbox/UW_Shean/COP/COP30_utqiagvik_lzw-adj_proj.tif"

plotter = StereoPlotter(
  directory, 
  stereo_directory,
  # Optional args:
  # dem_fn="my-custom-dem-name.tif",
  dem_gsd=1,
  # reference_dem=reference_dem,
)

### Hillshade with Details

Detailed hillshade with intersection error percentiles shown in inset.

In [ ]:
plotter.title = "Hillshade with details"

plotter.plot_detailed_hillshade(
  intersection_error_percentiles=[16, 50, 84],
  subset_km=1,
)

### Stereo Match Points

Locations of interest points used for stereo correlation.

In [ ]:
plotter.title="Stereo Match Points"

plotter.plot_match_points()

### Disparity (Meters)

Pixel offsets between stereo images, converted to ground distance in meters.

In [ ]:
plotter.title = "Disparity (meters)"

plotter.plot_disparity(
  unit="meters",
  quiver=True,
)

### Disparity (Pixels)

Pixel offsets between stereo images in original image pixel coordinates.

In [ ]:
plotter.title = "Disparity (pixels)"

plotter.plot_disparity(
  unit="pixels",
  quiver=True,
)

### DEM Results Summary

Multi-panel plot showing elevation, intersection error, and difference vs. reference DEM.

In [ ]:
plotter.title = "Stereo DEM Results"

plotter.plot_dem_results()

In [ ]:
# You can also use custom color limits for the DEM results plots:

# plotter.title = "Stereo DEM Results (forced color limits)"

# plotter.plot_dem_results(
#   el_clim=(0, 10),
#   ie_clim=(0, 0.2),
#   diff_clim=(-5, 5),
# )

## 7. ICESat-2 Altimetry Validation

Compare the ASP DEM with ICESat-2 ATL06-SR altimetry data from NASA's SlideRule API.

This section:
1. Requests ATL06-SR data for the DEM extent
2. Filters by processing level (all, ground, canopy, top-of-canopy)
3. Applies temporal filters around the acquisition date
4. Performs DEM-to-altimetry alignment using ASP's `pc_align`
5. Generates comparison plots and statistics

**Note**: Requires internet connection for SlideRule API access.

In [ ]:
import glob
from asp_plot.altimetry import Altimetry

In [ ]:
dem_fn = glob.glob(os.path.join(directory, "stereo*/*DEM_1m.tif"))[0]
aligned_dem_fn = glob.glob(os.path.join(directory, "stereo*/*DEM_1m*pc_align*.tif"))[0]

map_crs = "32604"  # UTM Zone 4N

ctx_kwargs = {
    "crs": f"EPSG:{map_crs}",
    "source": ctx.providers.Esri.WorldImagery,
    "attribution_size": 0,
    "alpha": 0.5,
}

In [ ]:
icesat = Altimetry(
  directory=directory,
  dem_fn=dem_fn,
  # If the aligned_dem_fn is not provided, it will be calculated automatically
  # using ASP's pc_align tool, if it is available in your PATH.
  aligned_dem_fn=aligned_dem_fn
)

### Request ATL06-SR Data

Fetch ICESat-2 data from SlideRule for multiple processing levels. Data is cached locally as parquet files.

In [ ]:
icesat.request_atl06sr_multi_processing(
    save_to_parquet=True,
)

# There are many more custom processing options available.
# See SlideRule's Parameter documentation for details:
# https://slideruleearth.io/web/rtd/user_guide/icesat2.html#parameters

# icesat.request_atl06sr_multi_processing(
#     res=10,
#     len=20,
#     ats=20,
#     cnt=5,
#     maxi=5,
#     save_to_parquet=True,
#     processing_levels=["ground"],
# )

In [ ]:
print("There are", icesat.atl06sr_processing_levels["all"].shape[0], "ATL06SR points available.")
print("There are", icesat.atl06sr_processing_levels_filtered["all"].shape[0], "filtered ATL06SR points available.")

### Filter ATL06-SR Data

Apply land cover and temporal filters to improve comparison quality.

In [ ]:
# Filter by ESA WorldCover to remove water points
icesat.filter_esa_worldcover(filter_out="water")

In [ ]:
print("After filtering out water, there are", icesat.atl06sr_processing_levels_filtered["ground"].shape[0], "filtered ground ATL06SR points available.")

In [ ]:
# Apply temporal filters around acquisition date (15/45/91 day windows + seasonal)
icesat.predefined_temporal_filter_atl06sr()

# You can also create user defined temporal filters with `generic_temporal_filter_atl06sr`:

# icesat.generic_temporal_filter_atl06sr(
#     select_years=[2021, 2022, 2023],
#     select_months=[1, 2, 3, 10, 11, 12],
#     select_days=[1, 2, 3, 4, 5, 6, 7, 27, 28, 29, 30, 31]
# )

In [ ]:
print("There are", icesat.atl06sr_processing_levels_filtered["ground_seasonal"].shape[0], "filtered ground seasonal ATL06SR points available.")

### ICESat-2 Temporal Distribution

Visualize the temporal distribution of ATL06-SR points colored by acquisition date.

In [ ]:
icesat.plot_atl06sr_time_stamps(
   key="ground",
   figsize=(15, 10),
   save_dir=None,
   fig_fn=None,
   **ctx_kwargs,
)

In [ ]:
icesat.plot_atl06sr_time_stamps(
   key="canopy",
   figsize=(15, 10),
   save_dir=None,
   fig_fn=None,
   **ctx_kwargs,
)

### ICESat-2 Spatial Distribution

Map view of ATL06-SR points colored by elevation and laser spot.

In [ ]:
icesat.plot_atl06sr(
    key="ground_seasonal",
    map_crs=map_crs,
    cmap="inferno",
    plot_dem=False,
    **ctx_kwargs
)

In [ ]:
icesat.plot_atl06sr(
    key="ground_45_day_pad",
    map_crs=map_crs,
    cmap="inferno",
    plot_dem=False,
    plot_beams=True,
    **ctx_kwargs
)

In [ ]:
icesat.plot_atl06sr(
    key="canopy_seasonal",
    map_crs=map_crs,
    cmap="inferno",
    plot_dem=True,
    plot_beams=True,
    **ctx_kwargs
)

### DEM vs. ICESat-2 Comparison

Map view showing DEM-altimetry differences before and after alignment.

In [ ]:
icesat.mapview_plot_atl06sr_to_dem(
    key="ground_15_day_pad",
    **ctx_kwargs,
)

In [ ]:
# Now for the pc_align aligned DEM version
icesat.mapview_plot_atl06sr_to_dem(
    key="ground_15_day_pad",
    plot_aligned=True,
    **ctx_kwargs,
)

In [ ]:
# Show a quick histogram
icesat.histogram(
    key="ground_seasonal",
    plot_aligned=True,
)

### PC Alignment Report

Perform `pc_align` with multiple temporal filters and compare results. This helps assess alignment consistency and chooses the best temporal filter.

In [ ]:
icesat.alignment_report(
    processing_level="ground",
    minimum_points=500,
    agreement_threshold=0.25,
    write_out_aligned_dem=True,
    min_translation_threshold=0.1,
    key_for_aligned_dem="ground_15_day_pad",
)

In [ ]:
icesat.alignment_report_df

In [ ]:
icesat.mapview_plot_atl06sr_to_dem(
    key="ground_15_day_pad",
    plot_aligned=True,
    **ctx_kwargs,
)

# Show a quick histogram
icesat.histogram(
    key="ground_seasonal",
    plot_aligned=True,
)

### WIP: Profile plots

Planned functionality

In [ ]:
# Collect only the coincident filtereded data again for profile plotting
# icesat.filter_atl06sr(
#     h_sigma_quantile=0.95,
#     mask_worldcover_water=True,
#     save_to_csv=False,
#     select_months=[4],
#     select_years=[2022],
# )

# icesat.plot_atl06sr(
#     title=f"Cleaned beam strengths (n={icesat.atl06sr_filtered.shape[0]})",
#     filtered=True,
#     plot_beams=True,
#     plot_dem=False,
#     map_crs=map_crs,
#     **ctx_kwargs
# )

# icesat.plot_atl06sr_dem_profiles(title="Profiles", only_strong_beams=True)

## 8. CSM Camera Model Analysis

Compare original and optimized CSM (Community Sensor Model) camera models from `bundle_adjust` or `jitter_solve`.

Visualizes:
- Position differences (X, Y, Z) along the satellite orbit
- Orientation angle differences (roll, pitch, yaw)
- Camera footprints on a map

**Note**: This example uses different test data (Salar de Uyuni, Bolivia) to demonstrate jitter correction.

In [ ]:
from asp_plot.csm_camera import csm_camera_summary_plot

In [ ]:
map_crs = "32619" # UTM 19S (for Uyuni)
title = "Jitter correction (Uyuni), Less Constrained"

ctx_kwargs = {
    "crs": f"EPSG:{map_crs}",
    "source": ctx.providers.Esri.WorldImagery,
    "attribution_size": 0,
    "alpha": 0.5,
}

# First set of cameras (scene 1)
original_camera = "../../tests/test_data/jitter/uyuni/csm-104001001427B900.r100.adjusted_state.json"
optimized_camera = "../../tests/test_data/jitter/uyuni/jitter_solved_run-csm-104001001427B900.r100.adjusted_state.json"
cam1_list = [original_camera, optimized_camera]

# Second set of cameras (scene 2)
original_camera = "../../tests/test_data/jitter/uyuni/csm-1040010014761800.r100.adjusted_state.json"
optimized_camera = "../../tests/test_data/jitter/uyuni/jitter_solved_run-csm-1040010014761800.r100.adjusted_state.json"
cam2_list = [original_camera, optimized_camera]

### CSM Camera Comparison (Basic)

Basic comparison without trimming or scaling.

In [ ]:
csm_camera_summary_plot(
    cam1_list,
    cam2_list,
    map_crs=map_crs,
    title=title,
    trim=False,
    figsize=(20, 15),
    add_basemap=True,
    **ctx_kwargs
)

### CSM Camera Comparison (Advanced)

With trimming, log scaling, and percentile-based color limits to emphasize patterns.

In [ ]:
csm_camera_summary_plot(
    cam1_list,
    cam2_list,
    map_crs=map_crs,
    title=title,
    trim=True,
    shared_scales=True,
    log_scale_positions=True,
    log_scale_angles=True,
    upper_magnitude_percentile=95,
    figsize=(20, 15),
    add_basemap=True,
    **ctx_kwargs
)

### Single Camera Comparison

You can also analyze a single satellite by passing only `cam1_list`.

In [ ]:
csm_camera_summary_plot(
    cam1_list,
    map_crs=map_crs,
    title=title,
    trim=True,
    shared_scales=True,
    log_scale_positions=True,
    log_scale_angles=True,
    upper_magnitude_percentile=95,
    figsize=(20, 15),
    add_basemap=True,
    **ctx_kwargs
)